## !! The final result should be only a runnable .py file !!

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 0. Data Pre-processing

In [4]:
data_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # turn the graph to single color channel
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # normalize
])

train_dataset = datasets.ImageFolder(
    '../dataset/train', transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.ImageFolder(
    '../dataset/test', transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


# 1. Model

In [5]:
class EmotionCNN(nn.Module):
    def __init__(self, num_classes):
        super(EmotionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 24 * 24, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = x.view(-1, 16 * 24 * 24)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

# initialize model, loss-function and optimizer
model = EmotionCNN(num_classes=7)  # FER-2013 has 7 emotion class
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
# select device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("using cuda")
#elif torch.backends.mps.is_available():
#    device = torch.device("mps")
#    print("using mac mps")
else:
    device = torch.device("cpu")
    print("using cpu")

using cpu


In [13]:
# training model
num_epochs = 10
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(
        f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")


Epoch 1/10, Loss: 0.9517489491167472
Epoch 2/10, Loss: 0.9015140030325653
Epoch 3/10, Loss: 0.8558426953370959
Epoch 4/10, Loss: 0.8050724761788723
Epoch 5/10, Loss: 0.7552981721133591
Epoch 6/10, Loss: 0.7103111799018155
Epoch 7/10, Loss: 0.6646988815347973
Epoch 8/10, Loss: 0.6205421885297134
Epoch 9/10, Loss: 0.581139644495894
Epoch 10/10, Loss: 0.5463671005250086


In [14]:
# evaluate model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1) # predicted is the emotion index
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy}%")


Test Accuracy: 47.561994984675394%
